In [3]:
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from os import makedirs
import os.path, time, re

#이미 처리한 파일인지 확인하기 위한 변수
proc_files = []

#HTML 내부에 있는 링크를 추출하는 함수
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup/select("link[re='stylesheet']")
    links += soup.select("a[href]")
    result = []

    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result

# 파일을 다운받고 저장하는 함수
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path
    if re.search(r"/$", savepath):
        savepath += "index.html"
    savedir = os.path.dirname(savepath)
    # 모두 다운됐는지 확인
    if os.path.exists(savepath): return savepath
    # 다운받을 폴더 생성
    if not os.path.exists(savedir):
        print("mkdir=", savedir)
        makedirs(savedir)
    # 파일 다운받기
    try:
        print("download=", url)
        urlretrieve(url, savepath)
        time.sleep(1)
    except:
        print("다운 실패: ", url)
        return None

#HTML을 분석하고 다운받는 함수
def analyze_html(url, root_url):
    savepath = download_file(url)
    if savepath is None: return
    if savepath in proc_files: # 이미 처리됐다면 실행하지 않음
        proc_files[savepath] = True
        print("analyze_html=", url)
    #링크 추출
    html = open(savepath, 'r', encoding='utf-8').read()
    links = enum_links(html, url)

    for link_url in links:
        #링크가 루트 이외의 경로를 나타낸다면 무시
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url): continue
        #HTML이라면
        if re.search(r".(html|htm)$", link_url):
            #재귀적으로 HTML 파일 분석하기
            analyze_html(link_url, root_url)
        # 기타파일
        download_file(link_url)

if __name__ =="__main__":
    # url에 있는 모든 것 다운받기
    url = "https://data.seoul.go.kr/dataList/datasetView.do?infId=OA-2266&srvType=S&serviceKind=1&currentPageNo=1"
    analyze_html(url, url)

mkdir= ./data.seoul.go.kr/dataList
download= https://data.seoul.go.kr/dataList/datasetView.do?infId=OA-2266&srvType=S&serviceKind=1&currentPageNo=1
다운 실패:  https://data.seoul.go.kr/dataList/datasetView.do?infId=OA-2266&srvType=S&serviceKind=1&currentPageNo=1


In [4]:
!ls

1_1_01_download-ip.py            1_2_06_bs-usd.py
1_1_02_download-forecast.py      1_3_01_books.html
1_1_03_download-forecast-argv.py 1_3_01_sel-book.py
1_2_01_bs-test1.py               1_3_01_sel-dongju.ipynb
1_2_02_bs-link.py                1_4_01_cr-getall.ipynb
1_2_03_bs-test2.py               data.seoul.go.kr
1_2_04_bs-forecast.py            docs.python.org
1_2_05_bs-select.py
